<a href="https://colab.research.google.com/github/clionelove123/temp_test/blob/main/Weather_Prediction_20230922_noted.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from numpy import asarray
import tensorflow as tf
import pandas as pd
import PIL
from PIL import Image
import random
import time

In [ ]:
# Google 드라이브에 있는 파일을 접속하게 해주는 코드 → 여기서 구글 드라이브는 제 계정입니다.
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Google 드라이브에 있는 데이터를 가져오는 코드입니다.
# csv_name_list / img_name_list는 해당 데이터 이름입니다
# csv_list / csv_file_name 변수는 해당 데이터의 파일 이름을 array화함으로써, 쉽게 파일을 읽어올 수 있게 하는 작업입니다.

csv_name_list_file='/content/drive/MyDrive/Colloquium/2020_data_csv.csv'
img_name_list_file='/content/drive/MyDrive/Colloquium/2020_data_img.csv'

csv_folder_name='/content/drive/MyDrive/Colloquium/2020_data_csv'
img_folder_name='/content/drive/MyDrive/Colloquium/2020_data_img'


csv_name_list=pd.read_csv(csv_name_list_file,encoding='cp949')

csv_list=csv_name_list['name']

csv_file_name=csv_folder_name+'/'+csv_list[0]
csv_file_name_next=csv_folder_name+'/'+csv_list[1]

csv_file=np.loadtxt(csv_file_name,delimiter=',')
csv_file_next=np.loadtxt(csv_file_name_next,delimiter=',')


In [ ]:
# 같은 작업을 img에 대해서도 수행합니다.

img_name_list=pd.read_csv(img_name_list_file,encoding='cp949')

img_list=img_name_list['name']

img_file_name=img_folder_name+'/'+img_list[0]

temp_image=Image.open(img_file_name)

temp_image_array=asarray(temp_image)

In [ ]:
# CNN 모델을 정의합니다 (with keras)
# 신경망의 구조는 이미지를 3개의 Convolution / MaxPool Layer를 이용하여 128개의 layer (fc_layer_1)으로 가져오고
# fc_layer_1과 data_input (이건 24시간 x 4종류 데이터 = 96개 숫자, 후단의 input 시 형태 재구성) 을 붙여주고 (h_concate1 = tf.keras.layers.Concatenate 부분)
# fc_layer_2 / fc_layer_3는 1024개의 노드를 가진 Fully Connected Layer로써, 1024개의 feature로 각각 정리하는 부분입니다. (CNN의 FC 부분과 동일)
# 여기서 해당 node들이 binary (true or false)가 아닌 linear value를 갖는 것이 맞다고 판단하여,
# 이는 우리가 binary한 feature에 의거하여 classification하는게 아닌, 숫자를 가져오는 것이 목적이기 때문입니다.
# ex) 강아지 이미지를 분류한다고 하면, 어떠한 feature는 눈이 충분히 똘망똘망한가? 와 같은 형태가 되어, 똘망똘망 합니다. 아닙니다 만 가려내어 판단하게 되는데, 이것이 activation을 활용하는 방식입니다.
# 우리는 어제의 보정 온도 수치는 27이므로, 내일 13시 기압은 27 입니다. 와 같은형태로 판단해야 하므로, activation 부분을 none으로 처리하였습니다. → 가중치를 곱해 더한 값을 그대로 활용
# fc_layer_4는 fc_layer_3의 1024개 피쳐를 활용하여 최종적으로 96개의 linear한 값을 출력하도록 만든 Fully Connected Layer 입니다. 마찬가지로 activation function은 활용하지 않았습니다.


class CNN(tf.keras.Model):
  def __init__(self):
    super(CNN, self).__init__()

    self.conv_layer_1 = tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=1, padding='same', activation='relu')
    self.pool_layer_1 = tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=2)

    self.conv_layer_2 = tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=1, padding='same', activation='relu')
    self.pool_layer_2 = tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=2)

    self.conv_layer_3 = tf.keras.layers.Conv2D(filters=32, kernel_size=3, strides=1, padding='same', activation='relu')
    self.pool_layer_3 = tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=2)

    self.flatten_layer = tf.keras.layers.Flatten()
    self.fc_layer_1 = tf.keras.layers.Dense(128, activation=None)

    self.fc_layer_2 = tf.keras.layers.Dense(1024, activation=None)
    self.fc_layer_3 = tf.keras.layers.Dense(1024, activation=None)
    self.fc_layer_4 = tf.keras.layers.Dense(96, activation=None)


  def call(self,img_input,data_input):
#    x_image = tf.reshape(img_input, [-1, 896, 896, 3])
#    x_data=tf.reshape(data_input, [-1, 96])
#    x_date=tf.reshape(date_input, [-1, 2])
    x_image=img_input
    x_data = data_input
#    x_date = date_input


    h_conv1 = self.conv_layer_1(x_image)
    h_pool1 = self.pool_layer_1(h_conv1)
    h_conv2 = self.conv_layer_2(h_pool1)
    h_pool2 = self.pool_layer_2(h_conv2)
    h_conv3 = self.conv_layer_3(h_pool2)
    h_pool3 = self.pool_layer_3(h_conv3)
    h_flat = self.flatten_layer(h_pool3)
    h_fc1 = self.fc_layer_1(h_flat)

#    h_concate1 = tf.keras.layers.Concatenate(axis=-1)([h_fc1,x_data,x_date])
    h_concate1 = tf.keras.layers.Concatenate(axis=-1)([h_fc1,x_data])

    h_fc2 = self.fc_layer_2(h_concate1)
    h_fc3 = self.fc_layer_3(h_fc2)
    h_fc4 = self.fc_layer_4(h_fc3)
#    h_fc4 = self.fc_layer_4(h_fc2)

#    h_convtrans_pre = self.conv_2d_trans_pre(h_fc2)
#    h_convtrans_1 = self.conv_2d_trans_1(h_convtrans_pre)
#    h_convtrans_2 = self.conv_2d_trans_2(h_convtrans_1)
#    h_convtrans_3 = self.conv_2d_trans_3(h_convtrans_2)
#    h_convtrans_4 = self.conv_2d_trans_4(h_convtrans_3)
#    h_convtrans_final = self.conv_2d_trans_final(h_convtrans_4)
#    return h_convtrans_final
    return h_fc4


In [ ]:
# 손실함수를 정의합니다. 손실함수는 최소화 시키고자 하는 값입니다.
# 우리는 96개의 값을 예측하고자 하는 것이 목표이며, 내일의 96개의 값과 비슷하게 맞추는 것이 목표입니다.
# 이에, 96개 값을 각각 차이를 비교한 후, 제곱을 시켜 주었습니다. (math.power)
# 이후 차이값에 제곱을 한 96개 값을 모두 더한 값을 손실함수 결과값으로 지정하였습니다.
# 이는 기본적으로 MSE (Mean Squared Error)를 구하는 것과 동일한 방식입니다.
# 다만 1개의 데이터에서 1개의 에러가 아닌 96개의 에러가 발생하여, 이를 더하는 과정이 추가된것 뿐입니다

@tf.function
def loss_cal(predict,next_day):
  a=tf.math.pow(predict-next_day,2)
  b=tf.reduce_sum(a)
  return b

# 최적화를 위한 Adam 옵티마이저를 정의합니다.
optimizer = tf.optimizers.Adam(1e-4)

# 학습을 진행하는 함수입니다. 보통 tensorflow는 model을 정의하고 fit함수를 주로 활용하지만, 저는 신경망 수정을 위해 아래와 같이 따로 정의하였습니다.
# 역할은 동일합니다. loss함수의 gradient 값을 계산하고, 이를 토대로 신경망의 각 가중치 / 바이어스 (Weight / Bias)값을 갱신해 주는 함수입니다.

@tf.function
def train_step(model,img_input,data_input,data_next):
  with tf.GradientTape() as tape:
    y_pred = model(img_input,data_input)
    loss = loss_cal(y_pred,data_next)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

In [ ]:
# Convolutional Neural Networks(CNN) 모델을 선언합니다.
CNN_model = CNN()

#아래 부분은 data 처리를 시험적으로 해보는 부분입니다.

#img_input=tf.random.normal([1,896,896,3],-50,50,tf.float32)
img_input=tf.convert_to_tensor(temp_image_array,dtype=tf.float32)
img_input=tf.reshape(img_input,[1,896,896,3])

data_input=tf.convert_to_tensor(csv_file,dtype=tf.float32)
#data_input=tf.random.normal([1,24,4],-50,50,tf.float32)

date_input=tf.random.normal([1,1,2],-50,50,tf.float32)

data_next=tf.convert_to_tensor(csv_file_next,dtype=tf.float32)
#data_next=tf.random.normal([1,24,4],-50,50,tf.float32)

data_next_pre=tf.reshape(data_next, [-1, 96])

In [ ]:
#아래 부분은 data 처리를 시험적으로 해보는 부분입니다.
#img_input=tf.random.normal([1,896,896,3],0,255,tf.float32)
date_input_temp=[0.0,0.0]
date_input_pre=tf.convert_to_tensor(date_input_temp,dtype=tf.float32)
date_input=tf.reshape(date_input_pre, [-1, 2])

In [ ]:
# 적용하는 날수와 학습 진행횟수를 결정합니다.
# 우선은 1 ~ 350일에 대해서 진행하며, 총 15회 순환합니다.
no_day=350
no_epoch=15

# 실제 학습을 진행하는 부분입니다.

for i in range(no_day*no_epoch):
  now=time
  k=random.randrange(0,no_day)
#  for k in range(no_day):
  csv_file_name=csv_folder_name+'/'+csv_list[k]
  csv_file_name_next=csv_folder_name+'/'+csv_list[k+1]

  csv_file=np.loadtxt(csv_file_name,delimiter=',')
  csv_file_next=np.loadtxt(csv_file_name_next,delimiter=',')
  data_input_pre=tf.convert_to_tensor(csv_file,dtype=tf.float32)
  data_input=tf.reshape(data_input_pre, [-1, 96])
  data_next_pre=tf.convert_to_tensor(csv_file_next,dtype=tf.float32)
  data_next=tf.reshape(data_next_pre, [-1, 96])

  img_file_name=img_folder_name+'/'+img_list[k]
  temp_image=Image.open(img_file_name)
  temp_image_array=asarray(temp_image)
  img_input=tf.convert_to_tensor(temp_image_array,dtype=tf.float32)
  img_input=tf.reshape(img_input,[1,896,896,3])

#  date_input=tf.random.normal([1,1,2],-50,50,tf.float32)
#  date_input=tf.convert_to_tensor(date_input_pre,dtype=tf.float32)

  train_step(CNN_model,img_input,data_input,data_next)

#  loss_temp=loss_print
  if i%no_day<1:
    loss_print = loss_cal(CNN_model(img_input,data_input),data_next)
    print(i,k,now.localtime().tm_hour,now.localtime().tm_min,now.localtime().tm_sec,loss_print)

loss_print = loss_cal(CNN_model(img_input,data_input),data_next)
print(i,k,loss_print)

0 294 6 3 57 tf.Tensor(8419196.0, shape=(), dtype=float32)
350 291 6 4 12 tf.Tensor(592.9961, shape=(), dtype=float32)
700 279 6 4 26 tf.Tensor(546.76495, shape=(), dtype=float32)
1050 326 6 4 41 tf.Tensor(588.8021, shape=(), dtype=float32)
1400 317 6 4 56 tf.Tensor(256.75858, shape=(), dtype=float32)
1750 57 6 5 11 tf.Tensor(231.57214, shape=(), dtype=float32)
2100 266 6 5 25 tf.Tensor(236.32982, shape=(), dtype=float32)
2450 223 6 5 40 tf.Tensor(304.71017, shape=(), dtype=float32)
2800 87 6 5 55 tf.Tensor(80.708145, shape=(), dtype=float32)
3150 38 6 6 10 tf.Tensor(141.17976, shape=(), dtype=float32)
3500 13 6 6 25 tf.Tensor(78.16213, shape=(), dtype=float32)
3850 148 6 6 39 tf.Tensor(93.77379, shape=(), dtype=float32)
4200 48 6 6 54 tf.Tensor(100.82944, shape=(), dtype=float32)
4550 183 6 7 9 tf.Tensor(158.18993, shape=(), dtype=float32)
4900 106 6 7 24 tf.Tensor(148.6767, shape=(), dtype=float32)
5249 342 tf.Tensor(46.925217, shape=(), dtype=float32)


In [ ]:
result_pre=np.zeros(no_day)

for k in range(no_day):
  csv_file_name=csv_folder_name+'/'+csv_list[k]
  csv_file_name_next=csv_folder_name+'/'+csv_list[k+1]

  csv_file=np.loadtxt(csv_file_name,delimiter=',')
  csv_file_next=np.loadtxt(csv_file_name_next,delimiter=',')
  data_input_pre=tf.convert_to_tensor(csv_file,dtype=tf.float32)
  data_next_pre=tf.convert_to_tensor(csv_file_next,dtype=tf.float32)
  data_input=tf.reshape(data_input_pre, [-1, 96])
  data_next=tf.reshape(data_next_pre, [-1, 96])

  img_file_name=img_folder_name+'/'+img_list[k]
  temp_image=Image.open(img_file_name)
  temp_image_array=asarray(temp_image)
  img_input=tf.convert_to_tensor(temp_image_array,dtype=tf.float32)
  img_input=tf.reshape(img_input,[1,896,896,3])

#  date_input=tf.random.normal([1,1,2],-50,50,tf.float32)
  date_input=tf.convert_to_tensor(date_input_pre,dtype=tf.float32)
  loss_print = loss_cal(CNN_model(img_input,data_input),data_next)
  result_pre[k]=loss_print

print(tf.reduce_sum(result_pre)/no_day)
print(result_pre)


tf.Tensor(142.75474414280484, shape=(), dtype=float64)
[  46.58016205   60.77753448   77.29792786   44.23731232  118.41116333
  870.17279053  160.47180176   73.633255     51.59705353   40.66537857
  106.38554382   60.41010666   65.0327301    66.82080078   61.44659424
   68.27671814   40.50237274   98.53358459  134.08944702   61.23526001
  183.91830444   48.01354599   41.45153809   80.22496796  367.11669922
  412.82244873  116.49187469   25.3164444    79.75110626   91.29704285
   43.49226379   81.32917023   81.03256989  174.35678101   56.23427582
   49.09899521  101.53767395  195.74491882   37.33421326  110.36329651
   26.37708664  129.21446228  157.90789795   69.54522705   94.31278992
   88.95100403  202.78869629   98.75730896   60.66326904   54.32847977
   89.34851074   62.81420898   78.94465637  228.80409241 1252.68005371
   80.66699219  170.59178162  152.2571106   152.251297     92.74365997
   48.45561218   46.34698486   62.03814697  101.87624359   67.62451172
   52.95350266  125.14

In [ ]:
k=random.randrange(1,no_day)

csv_file_name=csv_folder_name+'/'+csv_list[k]
csv_file_name_next=csv_folder_name+'/'+csv_list[k+1]
csv_file=np.loadtxt(csv_file_name,delimiter=',')
csv_file_next=np.loadtxt(csv_file_name_next,delimiter=',')
data_input_pre=tf.convert_to_tensor(csv_file,dtype=tf.float32)
data_next_pre=tf.convert_to_tensor(csv_file_next,dtype=tf.float32)
data_input=tf.reshape(data_input_pre, [-1, 96])
data_next=tf.reshape(data_next_pre, [-1, 96])

img_file_name=img_folder_name+'/'+img_list[k]
temp_image=Image.open(img_file_name)
temp_image_array=asarray(temp_image)
img_input=tf.convert_to_tensor(temp_image_array,dtype=tf.float32)
img_input=tf.reshape(img_input,[1,896,896,3])

#k_result=tf.reshape(CNN_model(img_input,data_input,date_input), [-1, 24,4])
#k_result=tf.reshape(CNN_model(img_input,data_input), [-1, 24,4])
k_result=CNN_model(img_input,data_input)
k_result_a=tf.math.round(k_result*10)/10

loss_print = loss_cal(k_result,data_next)
print(k,loss_print)

k_result_a=tf.reshape(k_result_a,[1,24,4])
data_next=tf.reshape(data_next,[1,24,4])
data_input=tf.reshape(data_input,[1,24,4])

print(k_result_a)
print(data_next)
print(data_input)


213 tf.Tensor(180.27646, shape=(), dtype=float32)
tf.Tensor(
[[[ 1.5 26.  13.6  0.1]
  [ 2.5 25.6 13.1  0.5]
  [ 2.1 25.4 13.6 -0.2]
  [ 3.9 26.4 12.8 -0.7]
  [ 5.4 25.4 12.5  0.4]
  [ 4.2 25.9 13.5 -1. ]
  [ 3.1 26.3 11.9  0.4]
  [ 1.6 28.4 13.5  0.3]
  [ 7.2 31.3 12.7 -0.2]
  [ 4.7 30.1 12.9 -0.1]
  [ 5.3 31.5 13.4 -1.7]
  [10.9 34.7 12.6  0.8]
  [13.5 34.7 12.5  1.4]
  [10.6 33.7 12.9 -0.1]
  [12.2 34.6 12.2 -0.1]
  [15.5 33.5 11.2 -0.1]
  [13.9 34.  11.7 -0.9]
  [13.9 33.7 12.9  0.4]
  [ 9.7 31.3 11.2  0.8]
  [11.1 29.9 12.2  0.7]
  [ 5.6 31.3 10.8  0. ]
  [ 3.9 29.5 11.9  0.3]
  [ 0.9 28.9 12.5 -1.4]
  [ 2.8 27.1 12.1 -0.5]]], shape=(1, 24, 4), dtype=float32)
tf.Tensor(
[[[ 2.  26.3 12.   0. ]
  [ 3.  26.4 11.7  0. ]
  [ 4.  26.6 10.9  0. ]
  [ 3.  26.9 10.7  0. ]
  [ 5.  26.7 10.7  0. ]
  [ 5.  26.8 10.8  0. ]
  [ 3.  26.9 10.8  0. ]
  [ 2.  28.2 10.8  0. ]
  [ 6.  28.7 11.2  0. ]
  [ 4.  28.9 11.5  0. ]
  [ 4.  30.  11.4  0. ]
  [11.  32.1 11.6  0. ]
  [12.  32.4 11.4  0. ]
  [1

In [ ]:
csv_name_list_file='/content/drive/MyDrive/Colloquium/2021_data_csv.csv'
img_name_list_file='/content/drive/MyDrive/Colloquium/2021_data_img.csv'

csv_folder_name='/content/drive/MyDrive/Colloquium/2021_data_csv'
img_folder_name='/content/drive/MyDrive/Colloquium/2021_data_img'


csv_name_list=pd.read_csv(csv_name_list_file,encoding='cp949')

csv_list=csv_name_list['name']

csv_file_name=csv_folder_name+'/'+csv_list[0]
csv_file_name_next=csv_folder_name+'/'+csv_list[1]

csv_file=np.loadtxt(csv_file_name,delimiter=',')
csv_file_next=np.loadtxt(csv_file_name_next,delimiter=',')

img_name_list=pd.read_csv(img_name_list_file,encoding='cp949')

img_list=img_name_list['name']

img_file_name=img_folder_name+'/'+img_list[0]

temp_image=Image.open(img_file_name)

temp_image_array=asarray(temp_image)

In [ ]:
result_pre=np.zeros(no_day)

for k in range(no_day):
  csv_file_name=csv_folder_name+'/'+csv_list[k]
  csv_file_name_next=csv_folder_name+'/'+csv_list[k+1]

  csv_file=np.loadtxt(csv_file_name,delimiter=',')
  csv_file_next=np.loadtxt(csv_file_name_next,delimiter=',')
  data_input_pre=tf.convert_to_tensor(csv_file,dtype=tf.float32)
  data_next_pre=tf.convert_to_tensor(csv_file_next,dtype=tf.float32)
  data_input=tf.reshape(data_input_pre, [-1, 96])
  data_next=tf.reshape(data_next_pre, [-1, 96])

  img_file_name=img_folder_name+'/'+img_list[k]
  temp_image=Image.open(img_file_name)
  temp_image_array=asarray(temp_image)
  img_input=tf.convert_to_tensor(temp_image_array,dtype=tf.float32)
  img_input=tf.reshape(img_input,[1,896,896,3])

#  date_input=tf.random.normal([1,1,2],-50,50,tf.float32)
  date_input=tf.convert_to_tensor(date_input_pre,dtype=tf.float32)
  loss_print = loss_cal(CNN_model(img_input,data_input),data_next)
  result_pre[k]=loss_print

print(tf.reduce_sum(result_pre)/no_day)
print(result_pre)

tf.Tensor(1542.272609209333, shape=(), dtype=float64)
[ 466.84848022  667.20306396  189.78277588  537.70800781 1769.90075684
 3579.90600586 2974.88305664 2045.78320312  967.76831055  550.00561523
 1421.62438965 2350.56835938  257.59320068 1846.46289062 2623.41015625
 1343.46411133 2833.24121094 1001.43579102  766.27490234 1114.50219727
  597.45123291 4160.86962891 1452.76196289 1869.99780273 1508.08459473
  437.77368164 3404.89624023 1216.08056641 1024.51086426  765.81896973
 1713.95410156 2594.25854492  327.11334229  580.71276855 1015.24560547
 1101.7310791   198.82957458 2218.29003906  143.84844971  307.82476807
  461.49804688  191.82983398  268.55609131 1782.37719727 2710.8425293
 2184.62792969 4217.92382812 1436.62426758 2648.09326172 4063.95605469
 3860.12084961 1380.41516113 2220.18554688  980.80859375 4007.84399414
 7607.40332031 4299.39453125 2651.6015625  4197.44238281 2909.34326172
 1470.31396484 1860.61462402  596.68493652 3026.77661133  899.82434082
  398.85369873  279.1329

In [ ]:
k=random.randrange(1,no_day)

csv_file_name=csv_folder_name+'/'+csv_list[k]
csv_file_name_next=csv_folder_name+'/'+csv_list[k+1]
csv_file=np.loadtxt(csv_file_name,delimiter=',')
csv_file_next=np.loadtxt(csv_file_name_next,delimiter=',')
data_input_pre=tf.convert_to_tensor(csv_file,dtype=tf.float32)
data_next_pre=tf.convert_to_tensor(csv_file_next,dtype=tf.float32)
data_input=tf.reshape(data_input_pre, [-1, 96])
data_next=tf.reshape(data_next_pre, [-1, 96])

img_file_name=img_folder_name+'/'+img_list[k]
temp_image=Image.open(img_file_name)
temp_image_array=asarray(temp_image)
img_input=tf.convert_to_tensor(temp_image_array,dtype=tf.float32)
img_input=tf.reshape(img_input,[1,896,896,3])

#k_result=tf.reshape(CNN_model(img_input,data_input,date_input), [-1, 24,4])
#k_result=tf.reshape(CNN_model(img_input,data_input), [-1, 24,4])
k_result=CNN_model(img_input,data_input)
k_result_a=tf.math.round(k_result*10)/10

loss_print = loss_cal(k_result,data_next)
print(k,loss_print)

k_result_a=tf.reshape(k_result_a,[1,24,4])
data_next=tf.reshape(data_next,[1,24,4])
data_input=tf.reshape(data_input,[1,24,4])

print(k_result_a)
print(data_next)
print(data_input)

112 tf.Tensor(1655.5028, shape=(), dtype=float32)
tf.Tensor(
[[[16.1 17.4 17.1  0.8]
  [14.5 17.3 17.2  0.6]
  [13.5 15.4 19.6  0.2]
  [16.1 15.6 18.9 -0.8]
  [14.5 16.9 19.6 -2.1]
  [15.2 18.5 19.5 -1.4]
  [12.7 17.3 20.1  0.8]
  [10.9 18.9 23.1 -0.5]
  [13.7 17.7 21.3  1. ]
  [12.5 16.9 21.5 -0.3]
  [10.7 16.3 23.4 -0.2]
  [12.  17.9 22.3 -1.4]
  [10.6 18.3 20.4 -1.3]
  [ 9.8 17.9 20.7 -0.1]
  [ 9.  20.2 21.4  1.9]
  [ 7.1 19.4 22.7  0.7]
  [ 6.9 18.1 17.2  0.4]
  [ 7.  19.3 20.7  0. ]
  [ 7.2 17.2 20.3  2.6]
  [ 6.2 19.7 21.8  0.4]
  [ 7.7 17.5 21.6 -1.1]
  [ 9.1 18.2 22.1  0.6]
  [ 4.3 17.9 22.3  0.4]
  [ 6.2 16.6 21.1 -1. ]]], shape=(1, 24, 4), dtype=float32)
tf.Tensor(
[[[20.  16.1 18.   0. ]
  [22.  15.9 17.4  0. ]
  [23.  15.8 17.1  0. ]
  [21.  15.7 16.7  0. ]
  [18.  15.7 16.9  0. ]
  [21.  15.8 16.8  0. ]
  [21.  15.8 17.4  0. ]
  [22.  16.1 18.1  0. ]
  [19.  16.3 19.   0. ]
  [20.  16.9 19.3  0. ]
  [19.  18.1 19.4  0. ]
  [18.  18.9 19.1  0. ]
  [21.  20.2 18.6  0. ]
  [1